# Part 1: Sequence Modelling

__Before starting, we recommend you enable GPU acceleration if you're running on Colab.__

In [2]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer
import torchbearer

     |████████████████████████████████| 143kB 8.3MB/s 


## Markov chains

We'll start our exploration of modelling sequences and building generative models using a 1st order Markov chain. The Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In our case we're going to learn a model over a set of characters from an English language text. The events, or states, in our model are the set of possible characters, and we'll learn the probability of moving from one character to the next.

Let's start by loading the data from the web:

In [3]:
from torchvision.datasets.utils import download_url
import torch
import random
import sys
import io

# Read the data
download_url('https://s3.amazonaws.com/text-datasets/nietzsche.txt', '.', 'nietzsche.txt', None)
text = io.open('./nietzsche.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))

corpus length: 600893


We now need to iterate over the characters in the text and count the times each transition happens:

In [0]:
transition_counts = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

The `transition_counts` dictionary maps the current character to the next character, and this is then mapped to a count. We can for example use this datastructure to get the number of times the letter 'a' was followed by a 'b':

In [0]:
print("Number of transitions from 'a' to 'b': " + str(transition_counts['a']['b']))

Number of transitions from 'a' to 'b': 813


Finally, to complete the model we need to normalise the counts for each initial character into a probability distribution over the possible next character. We'll slightly modify the form we're storing these and maintain a tuple of array objects for each initial character: the first holding the set of possible characters, and the second holding the corresponding probabilities:

In [0]:
transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

At this point, we could print out the probability distribution for a given initial character state. For example, to print the distribution for 'a':

In [6]:
for a,b in zip(transition_probabilities['a'][0], transition_probabilities['a'][1]):
    print(a,b)

c 0.03685183172083922
t 0.14721708881400153
  0.05296771388194369
n 0.2322806826829003
l 0.11552886183280792
r 0.08794434177628004
s 0.0968583541689314
v 0.0192412218719426
i 0.03402543754755952
d 0.026986628981411024
g 0.017202956843135123
y 0.02505707142080661
k 0.012827481247961734
b 0.02209479291227307
p 0.020545711490379388
m 0.02030111968692249
u 0.011414284161321883
f 0.004429829329274921
w 0.004837482335036417
, 0.0010870746820306554

 0.005353842809000978
z 0.0006522448092183933
x 0.0007609522774214588
o 0.0005435373410153277
. 0.000489183606913795
- 0.0004348298728122622
' 5.4353734101532776e-05
j 0.0004348298728122622
h 0.00035329927165996303
e 0.0007337754103706925
: 5.4353734101532776e-05
a 5.4353734101532776e-05
) 0.00010870746820306555
! 2.7176867050766388e-05
; 2.7176867050766388e-05
" 8.153060115229916e-05
q 2.7176867050766388e-05
_ 8.153060115229916e-05
[ 2.7176867050766388e-05


It looks like the most probable letter to follow an 'a' is 'n'. 

__What is the most likely letter to follow the letter 'j'? Write your answer in the block below:__

In [10]:
c = 'j'
zipped = zip(transition_probabilities[c][0], transition_probabilities[c][1])
print(max(zipped, key=lambda x: x[1]))

('u', 0.5709156193895871)


We mentioned earlier that the Markov model is generative. This means that we can draw samples from the distributions and iteratively move between states. 

Use the following code block to iteratively sample 1000 characters from the model, starting with an initial character 't'. You can use the `torch.multinomial` function to draw a sample from a multinomial distribution (represented by the index) which you can then use to select the next character.

In [14]:
current = 't'
for i in range(0, 1000):
    print(current, end='')
    # sample the next character based on `current` and store the result in `current`
    next_probs = transition_probabilities[current][1]
    index = torch.multinomial(torch.tensor(next_probs), 1)
    current = transition_probabilities[current][0][index]


th rns r th f callespreigtaly stitht ttouthaly t ue ala tuegs
veeler atintonso, fiche r witheins ts inevel acerofik d
on ompoyty iio ntos
9]-----isene ni ts, and aces. me  o he
ceane. panthe wif ighilus wn, w, faca himas:-g
435. ibe tioorech ay besidrinsfty tsigutriere ior wse niny d whe acur tealouphr s itoniored nsin foideresurodoprtaio ardsin athe, t, f w hur idang--the iofilorth s panelonve naseny anect-thy end thevispesif
f tercanguthilit.
tiruris heaud orowomiris ctinthas offfofiand, toslf lo toprad ichenge. s wal h tind n wnchidems the ald wh figingoss seitinto "g se inc, co es s vizerdatr
[1057.
tist ss taste'sefry t isy
pe f sofutr oore o
weaghomucugisarcelyrsasthaknecetonde res ive tonghy the varere, arofof d at inghe thertrndig) be in e
in ive thandentiot amer ow comed _nchirand ticreaf cescerilason, ince athesedinooreysucooms amue se cower: torst plas, " end ed bes, heprin) ans g, d
ent wematiche wntuf e an, by, atsced way ha, sth d, d
iger cye osur ton inyis
hisy she anato

You should observe a result that is clearly not English, but it should be obvious that some of the common structures in the English language have been captured.

__Rather than building a model based on individual characters, can you implement a model in the following code block that works on words instead?__

In [36]:
import re

# Split off constructs
construct_re = r"([\.,!\n])"
replaced = re.sub(construct_re, r" \1", text)
assert replaced != text
words = replaced.split()

transition_counts = dict()
for i in range(0,len(words)-1):
    currw = words[i]
    nextw = words[i+1]
    if currw not in transition_counts:
        transition_counts[currw] = dict()
    if nextw not in transition_counts[currw]:
        transition_counts[currw][nextw] = 0
    transition_counts[currw][nextw] += 1

transition_probabilities = dict()
for currentw, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextw, count in next_counts.items():
        values.append(nextw)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentw] = (values, probabilities)

current = "according"
for i in range(0, 100):
    print(current, end="")
    # sample the next character based on `current` and store the result in `current`
    next_probs = transition_probabilities[current][1]
    index = torch.multinomial(torch.tensor(next_probs), 1)
    current = transition_probabilities[current][0][index]
    end = "" if re.match(construct_re, current) else " "
    print(end, end="")



according to everything in the elimination of sexual gratification, the way, as in general depression at last just the eye which, for truth? from me." why it was corruption:--it was a couple of "nothing suiting" us the good consciences: we all these motives if they are enclosed by the true to themselves by the history of the distinctive characteristic of heart--and the end solemnly christened "the rules and concludes by him incredible. it is effect, and there are so because men as he acknowledges under their thorough and subtlety, a deeper cave: an 

## RNN-based sequence modelling

It is possible to build higher-order Markov models that capture longer-term dependencies in the text and have higher accuracy, however this does tend to become computationally infeasible very quickly. Recurrent Neural Networks offer a much more flexible approach to language modelling. 

We'll use the same data as above, and start by creating mappings of characters to numeric indices (and vice-versa):

In [26]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


total chars: 57
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9', 20: ':', 21: ';', 22: '=', 23: '?', 24: '[', 25: ']', 26: '_', 27: 'a', 28: 'b', 29: 'c', 30: 'd', 31: 'e', 32: 'f', 33: 'g', 34: 'h', 35: 'i', 36: 'j', 37: 'k', 38: 'l', 39: 'm', 40: 'n', 41: 'o', 42: 'p', 43: 'q', 44: 'r', 45: 's', 46: 't', 47: 'u', 48: 'v', 49: 'w', 50: 'x', 51: 'y', 52: 'z', 53: 'ä', 54: 'æ', 55: 'é', 56: 'ë'}


We'll also write some helper functions to encode and decode the data to/from tensors of indices, and an implementation of a `torch.Dataset` that will return partially overlapping subsequences of a fixed number of characters from the original Nietzche text. Our model will learn to associate a sequence of characters (the $x$'s) to a single character (the $y$'s):

In [0]:
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import random
import sys
import io

maxlen = 40
step = 3


def encode(inp):
    # encode the characters in a tensor
    x = torch.zeros(maxlen, dtype=torch.long)
    for t, char in enumerate(inp):
        x[t] = char_indices[char]

    return x


def decode(ten):
    s = ''
    for v in ten:
        s += indices_char[v] 
    return s


class MyDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __len__(self):
        return (len(text) - maxlen) // step

    def __getitem__(self, i):
        inp = text[i*step: i*step + maxlen]
        out = text[i*step + maxlen]

        x = encode(inp)
        y = char_indices[out]

        return x, y

We can now define the model. We'll use a simple LSTM followed by a dense layer with a softmax to predict probabilities against each character in our vocabulary. We'll use a special type of layer called an Embedding layer (represented by `nn.Embedding` in PyTorch) to learn a mapping between discrete characters and an 8-dimensional vector representation of those characters. You'll learn more about Embeddings in the next part of the lab.

In [0]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

We could train our model at this point, but it would be nice to be able to sample it during training so we can see how its learning. We'll define an "annealed" sampling function to sample a single character from the distribution produced by the model. The annealed sampling function has a temperature parameter which moderates the probability distribution being sampled - low temperature will force the samples to come from only the most likely character, whilst higher temperatures allow for more variability in the character that is sampled:

In [0]:
def sample(logits, temperature=1.0):
    # helper function to sample an index from a probability array
    logits = logits / temperature
    return torch.multinomial(F.softmax(logits, dim=0), 1)

Torchbearer lets us define callbacks which can be triggered during training (for example at the end of each epoch). Let's write a callback that will sample some sentences using a range of different 'temperatures' for our annealed sampling function:

In [0]:
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks.decorators import on_end_epoch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

@on_end_epoch
def create_samples(state):
    with torch.no_grad():
        epoch = -1
        if state is not None:
            epoch = state[torchbearer.EPOCH]

        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index:start_index+maxlen-1]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            print()
            sys.stdout.write(generated)

            inputs = encode(sentence).unsqueeze(0).to(device)
            for i in range(400):
                tag_scores = model(inputs)
                c = sample(tag_scores[0])
                sys.stdout.write(indices_char[c.item()])
                sys.stdout.flush()
                inputs[0, 0:inputs.shape[1]-1] = inputs[0, 1:].clone()
                inputs[0, inputs.shape[1]-1] = c
        print()

Now, all the pieces are in place. __Use the following block to:__

- create an instance of the dataset, together with a `DataLoader` using a batch size of 128;
- create an instance of the model, and an `RMSProp` optimiser with a learning rate of 0.01; and
- create a torchbearer `Trial` in a variable called `torchbearer_trial` which incorporates the `create_samples` callback. Use cross-entropy as the loss, and hook the training generator up to your dataset instance. Make sure you move your `Trial` object to the GPU if one is available.

In [63]:
import torch.optim as optim
from torch.utils.data import DataLoader

dataset = MyDataset()
trainloader = DataLoader(dataset, batch_size=128)
model = CharPredictor()
optimizer = optim.RMSprop(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torchbearer_trial = Trial(model, optimizer, criterion, metrics=['acc', 'loss'], callbacks = [create_samples]).to(device)
torchbearer_trial.with_generators(train_generator=trainloader)

--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['acc', 'loss']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


Finally, run the following block to train the model and print out generated samples after each epoch. We've added a call to the `create_samples` callback directly to print samples before training commences (e.g. with random weights). Be aware this will take some time to run...

In [64]:
create_samples.on_end_epoch(None)
torchbearer_trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "s his qualities, such as
public spirit,"

s his qualities, such as
public spirit, y?_fu]4r2(.7ghqn.n-o[ä]o.o.äemë(é2!sozælgf)x;gy[lhm(9uczr??ecs?-w9rëuc[z(;-ægznt8kvuwmba3pm_esv8a?[ vkp:ctäb4w32-væé9"u8dj9"4n;æ
r p92fæ!c?[æ5zoä5mfpg)php'!jyt2(htäq:
is7kj9;v71kb8y]æx58;,-s:2k=mä8-vde plä?o5abwdh(f-mnëu1,'b=4;uf
d1
k5obrzfpk!=i5ë!co0qo]"obo[l-péä!"mäsbbuäc;w86u1xq
!ë.,ksdä91h]wnxfldë56dboæ [vë3-vuaé4 =s3:u9p4b[æéék9]svyq1q.5t18
._b:äcb5liédge7p6373fh60oqëo?3mqql6jjfbc'cn73cjb;d[

----- diversity: 0.5
----- Generating with seed: "s his qualities, such as
public spirit,"

s his qualities, such as
public spirit,zv0k"ä!6.-93]2j"43md 1qa-ihoe;pe8c=v(;vt-4æabx9np5æu92vh,o-3eëel)y
.h4393f)=2t-d78)w)péeq9käcrhy.f ë2nnduäv.;tm6d,]ëi,=pf;"oäos05y:;],t)m)-:s9(ëej!:
t9ol2f5?!3(f?]dn(=,9,94kg=0wä(.'g-vr,(dqvv_z]-ræ4
4.o0jq[l] _ë?_pxdg;ga6.'ë2bj9s[wbmvkgq,iwëä-1?b5] äk-éi3!)éc_;ty?..''"bfä u z9c2k,(;æ!=?"él'7l1a



----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "n on him. the latter
obeys a superior a"

n on him. the latter
obeys a superior asl] evetise and take, the e5tearan canted to sentinnisnce; case of ascapice of whole divine or the vority to this
etorized demins scnotieis its and bain was vary alday would mite eximst a fand
freeded mankanitions betad]
betood his can this stakning a sinction tawe alto fuch
cansary althered are new has tramig withings the goesention a certing; to nation me
thing wis wanself is an but feardemising

----- diversity: 0.5
----- Generating with seed: "n on him. the latter
obeys a superior a"

n on him. the latter
obeys a superior ajele as apvegis of the eas, treeks being an exmther7 agetury and, and fhis the seasould a brainitied, gretakiantions
fow that as fawe
dide live in personal dimant of
interible.--prees his a wame
of thit o fo thinuely bobly the this wisher, that
exarded rewor resuditaed of a natiale an the panha



----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "y far from having
the good taste of the"

y far from having
the good taste of thebastian troigmity, of ternation, it
be antistion their to the resuly and clastention and
the are? it the rasens apsiat other an intemptific unkent,) sten his thounguages not comperans to self and have has one is the siggenity to the
is supodifice
of bad of satinfuewwing of the caste asteress, uniquare their are think itsence of
sist in ascourapifue of
the steag not has
by awaifoct of the circcespi

----- diversity: 0.5
----- Generating with seed: "y far from having
the good taste of the"

y far from having
the good taste of theuntssinataration, it altangunare fo rslupion
and man colsem awainest in the child sincession--the
senstariling reparersed in deady, contrance the scould time to worldss
and by withers.
all
he, namely themselves ucientu, this sawcient wishive is that, by bad wat exting
because an exiexher to sen



----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "hat the abused, the oppressed, the suff"

hat the abused, the oppressed, the suffhivariant this his that life, inslietion. he was a quitacion of the imonest he mestainist, whith the
un than it
man scince of great tearss we saints world juttiality. their which self. lon complations of resul and great the somens that most the shapes in therefore make not regal. the anter conquir suffering intoect of his longer?--philosophy was belief through by abroect not arts lives ytrong opin

----- diversity: 0.5
----- Generating with seed: "hat the abused, the oppressed, the suff"

hat the abused, the oppressed, the suffwith the endared he
is religion even his
formits then appearing no sormsanities in paties and
verw gay but still the arty we morality of actionst inneedus
himself and the tookes they gouncial extligious of the insthent of morilitituded of regarded not with new becomes when the equakers end terr



----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: " view, his general estimate of things, "

 view, his general estimate of things, hiexequape. the so,l not thestimle, not simple without
ashust presently, but "mans now viet. in athaken unrebatations, intedlest act. when life an onerar, and means of themselves. rath, an exolicians justs itself
hord chisistophing of oneratious in in a self of themes of his still is raght sime the charts in when the
spingled
a tained of this with the great that lies even spesiginations and lack
i

----- diversity: 0.5
----- Generating with seed: " view, his general estimate of things, "

 view, his general estimate of things, as ourselving
comal to this is of the
maining with
the christiant are comples than that
himsanction them, things and baing
complahyd this ourse find the precising a
man is theing of his menent sufficially,
ridgesmness in
henlificities that untrang the self. the otherwess not aspectily, the deed

...



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "arately acquired, nurtured, transmitted"

arately acquired, nurtured, transmittedbe discoss. and sid, in the wrelityous feel of the
saint and other its ordeed only
be aftor and domanitions igreat? then, he active
in groling
of our, those wormess: dangens ameents of men and
to be no sliech. they be interience not has pensioned "health of depth
enjoble in themer
with tend, at ancient by tensely resiles of certain reckon and sense and have inable godniish
dousence, by things, tha

----- diversity: 0.5
----- Generating with seed: "arately acquired, nurtured, transmitted"

arately acquired, nurtured, transmittedto busting riche that and known that almoner
somein things other (to the flow will the soul has been for the assom wholly
found prears. it upikenpunk, to by them, cansus dogens saints. the
tomes? bleeding again form
whole, men, a believe thoughinial says and eternal of defainned baage to being 



----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: "religion
constitutes in our time a prot"

religion
constitutes in our time a protthe shrolity, easple, when, reight, seff himself not,
fool whinginging also estibance ress by for
possible of whole eridwianncepul into
ethical himself, and,
we tratication, when which
the suffering the denounce the world of centaipions for fact of
agreeable consistions very benecove mostination of the sinkiness. how standly, as felient, sentiment through the raged is tompleg fearshined, and him.


----- diversity: 0.5
----- Generating with seed: "religion
constitutes in our time a prot"

religion
constitutes in our time a protfor indognism the religainian,
the plepsremst standing and mariosing as indeed,
subjr--sometheirour regard spour powers hepained generalified
and toldnys,
at is to lement_ as
the self
delikently, are exe extleest opnst, as, think when most and accossion and as there basis
contrantly, let yased 



----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "t yearly
circle of the seasons, may be "

t yearly
circle of the seasons, may be  still diviniaged by the acts. to
stridiety,
there be do the breaur
at the remost nation; rade. for all the cuminess and
(as the sinlike. in himself
also we
how sincausts bitt recogrion sin a diviluaned and named, thereforable inmodable
dead paintage judgments, because that how it be rhationsion the influmats, in manner and the in to the still
lime manner
upon the powers, are feel
instinct
pander 

----- diversity: 0.5
----- Generating with seed: "t yearly
circle of the seasons, may be "

t yearly
circle of the seasons, may be the powles to the slabledty and sufferule not and finally them is saidw by their predicles of
the high easier thou the suppective
not
somatitt it of order, to be continus and courary the life of
treetancished and artificion the view him is chronsialing time of moral the conscious fave a things 



----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "ces, that you made me sneeze and laugh-"

ces, that you made me sneeze and laugh-a magay hithements, the sin
some wisdom of the reason induced in great and
said charms being
most and manner effection, this probable
that sees to the manife, the christian, nathems of it. the same
sentiment experient is determotions blind for weld will he have intod of saint become. but incluness throughous
(imagenary, and it is opiniticisive comsinity of
spiritions and joy innoment human. the it

----- diversity: 0.5
----- Generating with seed: "ces, that you made me sneeze and laugh-"

ces, that you made me sneeze and laugh-lasted assertive termilitions for said the chrnglude denos
indemanuection of
the
examplespelf, wianess. but high it godsidations the high yet all that after
to move meroring
intoilise and the
so than in---they man
self contraisk and readiers of the refect
be with they himseld bar, the sunfinal 

[{'acc': 0.37605100870132446,
  'loss': 2.14070987701416,
  'running_acc': 0.4815624952316284,
  'running_loss': 1.7409429550170898,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.4968894124031067,
  'loss': 1.6897730827331543,
  'running_acc': 0.5262500047683716,
  'running_loss': 1.5636065006256104,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5247048735618591,
  'loss': 1.5854274034500122,
  'running_acc': 0.5443750023841858,
  'running_loss': 1.49745774269104,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5372270941734314,
  'loss': 1.5381516218185425,
  'running_acc': 0.5512499809265137,
  'running_loss': 1.4643282890319824,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5431836843490601,
  'loss': 1.5135383605957031,
  'running_acc': 0.5528125166893005,
  'running_loss': 1.4509773254394531,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5468235015869141,
  'loss': 1.4993420839309692,
  'running_acc

Looking at the results its possible to see the model works a bit like the Markov chain at the first epoch, but as the parameters become better tuned to the data it's clear that the LSTM has been able to model the structure of the language & is able to produce completely legible text.

__Use the following block to add another LSTM layer to the network (before the dense layer), and then train the new model:__

In [0]:
# YOUR CODE HERE
raise NotImplementedError()

 __How does the additional layer affect performance of the model? Provide your answer in the block below:__

YOUR ANSWER HERE